# Analyze results
Aug 28, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

In [5]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [6]:
# Read input data

fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
samples=np.load(fname,mmap_mode='r+')[:2000]
samples=samples[:,0,:,:]
print(samples.shape)

# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

(2000, 128, 128)


In [7]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [8]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201016_060239', '20201002_073628', '20201015_07254…

In [9]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

### Read loss data
with open (result_dir+'/metrics.pickle','rb') as f:
    metrics_dict=pickle.load(f)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201016_060239


## Plot Losses

In [10]:
df_metrics=pd.DataFrame([])
for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
df_metrics['step']=np.arange(df_metrics.shape[0])
df_metrics=df_metrics.dropna()

In [11]:
df_metrics.step.values.shape,df_metrics.G.values.shape

((15620,), (15620,))

In [12]:
df_metrics.head(10)

,Dreal,Dfake,Dfull,G,spec_chi,hist_chi,step
0,0.851537,0.613268,1.464805,509.700714,507.737044,0.002299,0
1,0.229608,1.116869,1.346478,375.079987,371.929156,0.002306,1
2,0.575670,0.339283,0.914953,280.365662,277.270744,0.002328,2
3,0.283477,0.409520,0.692997,268.956909,265.116492,0.002322,3
4,0.199796,0.375043,0.574839,465.777527,460.842369,0.002226,4
5,0.191341,0.206668,0.398009,273.031555,268.009108,0.002175,5
6,0.131419,0.291148,0.422567,42.732231,36.625227,0.002115,6
7,0.191246,0.150912,0.342159,123.741562,118.184803,0.002069,7
8,0.098609,0.313699,0.412308,805.126282,797.014153,0.002009,8
9,0.218321,0.098589,0.316909,826.624390,820.964844,0.002001,9


In [13]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('Dreal', 'Dfake', 'Dfull', 'G', 'spec_ch…

<function __main__.f_plot_metrics(df, col_list)>

In [14]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

1.7082531749110784e-05


,Dreal,Dfake,Dfull,G,spec_chi,hist_chi,step
6241,1.171352,0.249125,1.420476,20.413170,19.788598,0.000016,6241
6995,0.317193,0.945815,1.263008,56.915874,55.077480,0.000016,6995
6243,1.041202,0.312537,1.353739,10.942858,10.175337,0.000016,6243
6889,0.366328,0.853984,1.220312,85.016884,83.027335,0.000016,6889
6237,0.907393,0.398618,1.306011,49.116661,48.361364,0.000016,6237
6402,0.692783,0.582175,1.274958,14.213116,13.016200,0.000016,6402
6092,0.349629,1.035248,1.384877,34.699562,32.605757,0.000016,6092
6240,0.284226,1.181251,1.465477,21.313587,19.067615,0.000016,6240
6037,0.606635,0.524375,1.131009,2.996442,1.820755,0.000016,6037
6965,0.317914,0.810395,1.128309,40.747066,38.804046,0.000016,6965


In [18]:
### Read GAN results
# epoch,step=0,100
# fname='gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step)

step=6000
fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(result_dir+'/images/'+fname)[0]
fle
imgs=np.load(fle)
print(imgs.shape)

(128, 128, 128)


### Plot images

In [19]:
f_plot_grid(samples[100:118],cols=6,fig_size=(6,3))
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot Histogram

In [23]:
img_lst=[imgs]
f_compare_pixel_intensity(img_lst,label_lst=['imgs'],bkgnd_arr=samples,log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot spectrum

In [21]:
img_lst=[imgs]
f_compare_spectrum(img_lst,label_lst=['img'],bkgnd_arr=samples,log_scale=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Read stored chi-squares for images

In [129]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

chi1=df.quantile(q=0.2,axis=0)['chi_1']
chi_spec=df.quantile(q=0.2,axis=0)['chi_spec1']

df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_spec1','chi_1'])
# df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_1','chi_spec1']).head(5)
# df[df.chi_1<chi1].sort_values(by=['chi_1']).head(5)

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
210,6,10400,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.016605,0.000760,0.237058,0.254423,0.013059,8.466874,0.249465,2437.974680,14,2,"[1.0462612146768546, 1.0431065585966586, 0.284...","[0.004219244340959406, 0.003717733014430344, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[194267084.2283563, 74153.76104779728, 53394.5...","[317576.2794355429, 2770.8836579796603, 1445.0..."
90,2,4500,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.015343,0.016513,0.128876,0.160732,0.013799,8.055708,0.383665,4826.279731,5,0,"[1.176116208857838, 0.8597306252282902, 0.2698...","[0.004885021480753444, 0.004407815097079608, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[201595251.59615213, 86475.61848893017, 67405....","[375204.9043572703, 4165.933504164946, 2522.96..."
151,4,7500,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.018752,0.011356,0.199471,0.229578,0.010916,8.214697,0.396165,4047.289679,8,0,"[1.053203475990432, 1.0268868939174638, 0.3001...","[0.0050130863763587705, 0.0042308120199154664,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[195324433.86741328, 73768.11550347751, 54307....","[365326.3118080284, 2749.7375480852747, 1527.7..."
230,7,11400,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.044198,0.011017,0.169687,0.224901,0.036409,7.624178,0.427645,4646.836524,8,0,"[1.003139189755738, 1.104493022547646, 0.30108...","[0.003928279300521224, 0.003547343089748456, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[192939206.56214392, 70800.36232194304, 55277....","[284235.6149629534, 2575.116014526812, 1500.04..."
154,4,7650,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.001816,0.009035,0.258360,0.269212,0.000446,8.624604,0.470431,4775.420217,8,0,"[1.111552052967242, 0.9628127935297268, 0.2733...","[0.0052701422447912535, 0.004338296937168125, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[198088743.53374308, 72282.53611706197, 53347....","[387667.47840874677, 3169.8304968106336, 1432...."
115,3,5750,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.014801,0.008734,0.169518,0.193053,0.013247,7.573419,0.470454,3430.213482,6,0,"[1.182708283310914, 0.8657745845231275, 0.2575...","[0.003936454212797833, 0.003551956513323594, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[201477266.04165104, 68901.13493029773, 54715....","[304486.3913467814, 2814.923106934771, 1508.99..."
183,5,9050,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.012678,0.008548,0.242459,0.263685,0.008818,8.299217,0.482520,4640.737956,6,1,"[1.0601848311959334, 1.0254149949991298, 0.288...","[0.005225979539140182, 0.004821853095797746, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[195448954.18082273, 78305.81490946561, 56808....","[364762.63073185313, 3234.8035417550245, 1488...."
361,11,17950,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.050875,0.005277,0.189200,0.245353,0.043759,9.294178,0.491671,4979.495845,15,1,"[0.9905750853028858, 1.1194710797876355, 0.297...","[0.005958888176978793, 0.0046286505241318705, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[191612292.2708433, 77132.96705006063, 54185.3...","[423196.91991115944, 3574.532456159841, 1598.4..."
329,10,16350,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.036870,0.005622,0.209915,0.252406,0.034226,8.463343,0.504373,5787.392751,8,1,"[1.009712532524132, 1.1065007524698294, 0.2866...","[0.005625315411291972, 0.005174667310457091, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[192980273.547843, 77677.0487332493, 53495.077...","[367313.1909402773, 3265.360958891067, 1431.46..."
101,3,5050,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.003190,0.007935,0.114081,0.125206,0.001330,7.965937,0.505147,2557.453798,7,0,"[1.124699059049538, 0.93292

In [111]:
df[df.step==8000]

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
160,5,8000,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.001275,0.001343,0.544036,0.546653,0.00048,9.996117,0.384505,4884.080293,15,3,"[1.1173002946261748, 0.947256016600544, 0.2708...","[0.005825166400631041, 0.004291348889876834, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[197700198.73752385, 91077.45213577151, 62337....","[456146.6782676089, 3865.9732134400065, 1947.6..."
